In [ ]:
%pwd

'/workspace'

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from tqdm import tqdm

from openai import OpenAI

client = OpenAI(api_key="")

In [ ]:
TEH_i_l = load_dataset("AlanYky/hate-with-instruction-with-label")
TEO_i_l = load_dataset("AlanYky/offensive-with-instruction-with-label")
climate_i_l = load_dataset("AlanYky/climate-with-instruction-with-label")
SUBJ_i_l = load_dataset("AlanYky/subjective-with-instruction-with-label")

TEH_i_s = load_dataset("AlanYky/hate-with-instruction-with-symbol")
TEO_i_s = load_dataset("AlanYky/offensive-with-instruction-with-symbol")
climate_i_s = load_dataset("AlanYky/climate-with-instruction-with-symbol")
SUBJ_i_s = load_dataset("AlanYky/subjective-with-instruction-with-symbol")

TEH_n = load_dataset("AlanYky/hate-no-instruction-with-symbol")
TEO_n = load_dataset("AlanYky/offensive-no-instruction-with-symbol")
climate_n = load_dataset("AlanYky/climate-no-instruction-with-symbol")
SUBJ_n = load_dataset("AlanYky/subjective-no-instruction-with-symbol")

TEH_f = load_dataset("AlanYky/hate-flipped")
TEO_f = load_dataset("AlanYky/offensive-flipped")
climate_f = load_dataset("AlanYky/climate-flipped")
SUBJ_f = load_dataset("AlanYky/subjective-flipped")

In [ ]:
data = load_dataset("AlanYky/big-bench-list-function-turing-330")
modify_the_list = data.filter(lambda example: example['category'] == 'modify_the_list')
remove_elements = data.filter(lambda example: example['category'] == 'remove_elements')
input_independent = data.filter(lambda example: example['category'] == 'input_independent')
add_elements = data.filter(lambda example: example['category'] == 'add_elements')
miscellaneous = data.filter(lambda example: example['category'] == 'miscellaneous')
Turing = data.filter(lambda example: example['category'] == 'Turing')

In [ ]:
big_bench_prompt = "Apply a function to the final input list to generate the output list. Use any preceding inputs and outputs as examples."

def create_prompt(prompt, big_bench=False, big_bench_prompt=""):
  if big_bench:
    return f"Instruct: {big_bench_prompt}\n{prompt}\nOutput:"
  else:
    return f"Instruct: {prompt}\nOutput:"


def extract_answer(text):
        answer_idx = text.find("Output:")
        if answer_idx != -1:
            return text[answer_idx:].strip()
        return text

def create_messages(prompt, big_bench=False, big_bench_prompt=""):
    if big_bench:
       return [{"role": "user", "content": f"Instruct: {big_bench_prompt}\n{prompt}\nOutput:"}]
    else:
       return [{"role": "user", "content": f"Instruct: {prompt}\nOutput:"}]


def compare_models_on_dataset(dataset, num_data, eval_tokenizer, base_model, instruct_model, symbol_model, file_name, max_new_tokens=100):
    base_model.eval()
    instruct_model.eval()
    symbol_model.eval()
    samples = dataset['train'].shuffle(seed=43).select(range(num_data))
    inputs, targets, base_generations, instruct_generations, symbol_generations = [], [], [], [], []
    count = 0

    for example in tqdm(samples):
        #print(f"======Evaluating sample {count}")
        prompt = create_prompt(example['inputs'])
        model_input = eval_tokenizer(prompt, return_tensors="pt").to("cuda")
        #print(prompt)

        with torch.no_grad():
            base_gen = extract_answer(eval_tokenizer.decode(base_model.generate(**model_input, max_new_tokens=max_new_tokens)[0], skip_special_tokens=True))
            #print("Phi2-base================")
            #print(base_gen)

            instruct_gen = extract_answer(eval_tokenizer.decode(instruct_model.generate(**model_input, max_new_tokens=max_new_tokens)[0], skip_special_tokens=True))
            #print("Phi2-instruct================")
            #print(instruct_gen)

            symbol_gen = extract_answer(eval_tokenizer.decode(symbol_model.generate(**model_input, max_new_tokens=max_new_tokens)[0], skip_special_tokens=True))
            #print("Phi2-symbol================")
            #print(symbol_gen)


        # Append the results
        inputs.append(example['inputs'])
        targets.append(example['target'])
        base_generations.append(base_gen)
        instruct_generations.append(instruct_gen)
        symbol_generations.append(symbol_gen)

    df = pd.DataFrame({
        'Input': inputs,
        'Target': targets,
        'Base Model Generation': base_generations,
        'Instruction Model Generation': instruct_generations,
        'Symbol Model Generation': symbol_generations
    })

    df.to_csv(f'{file_name}.csv', index=False)

    return df

def model_eval(dataset, num_data, eval_tokenizer, symbol_model, file_name, max_new_tokens=50, big_bench=False, big_bench_prompt=""):
    symbol_model.eval()
    samples = dataset['train'].shuffle(seed=43).select(range(num_data))
    inputs, targets, symbol_generations = [], [], []

    for example in tqdm(samples):
        prompt = create_prompt(example['inputs'], big_bench, big_bench_prompt)
        model_input = eval_tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            symbol_gen = extract_answer(eval_tokenizer.decode(symbol_model.generate(**model_input, max_new_tokens=max_new_tokens)[0], skip_special_tokens=True))


        # Append the results
        inputs.append(example['inputs'])
        targets.append(example['target'])
        symbol_generations.append(symbol_gen)

    df = pd.DataFrame({
        'Input': inputs,
        'Target': targets,
        'Symbol Model Generation': symbol_generations
    })

    df.to_csv(f'{file_name}.csv', index=False)

    return df

def model_eval_m(dataset, eval_tokenizer, model, file_name, max_new_tokens=50, num_data=25, big_bench=False, big_bench_prompt=""):
    mistral_model.eval()
    samples = (dataset['train'].shuffle(seed=43))

    # Use all data during big_bench eval
    if not big_bench: samples = samples.select(range(num_data))

    inputs, targets, generations = [], [], []

    for example in tqdm(samples):
        model_input = eval_tokenizer.apply_chat_template(create_messages(example['inputs'], big_bench=big_bench, big_bench_prompt=big_bench_prompt), return_tensors="pt").to("cuda")

        with torch.no_grad():
            generated_ids = model.generate(model_input, max_new_tokens=max_new_tokens, do_sample=True)

            generated = extract_answer(eval_tokenizer.batch_decode(generated_ids)[0])
            # print(generated[:30])

        inputs.append(example['inputs'])
        if big_bench:
            targets.append(example['targets'])
        else:
            targets.append(example['target'])
        generations.append(generated)

    df = pd.DataFrame({
        'Input': inputs,
        'Target': targets,
        'Mistral Model Generation': generations
    })

    df.to_csv(f'{file_name}.csv', index=False)

    return df


def model_eval_gpt3(dataset, file_name, max_new_tokens=50, num_data=25, big_bench=False, big_bench_prompt=""):
    samples = (dataset['train'].shuffle(seed=43))
    # Use all data during big_bench eval
    if not big_bench: samples = samples.select(range(num_data))
    inputs, targets, generations = [], [], []

    for example in tqdm(samples):
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            temperature=0,
            max_tokens=max_new_tokens,
            messages=create_messages(example['inputs'], big_bench=big_bench, big_bench_prompt=big_bench_prompt))

        generated = extract_answer(completion.choices[0].message.content)

        inputs.append(example['inputs'])
        if big_bench:
            targets.append(example['targets'])
        else:
            targets.append(example['target'])
        generations.append(generated)

    df = pd.DataFrame({
        'Input': inputs,
        'Target': targets,
        'GPT-3.5 Generation': generations
    })

    df.to_csv(f'{file_name}.csv', index=False)

    return df

# Evaluation using GPT-3.5

In [ ]:
eval_sets = [
    TEH_i_l, TEO_i_l, climate_i_l, SUBJ_i_l,
    TEH_i_s, TEO_i_s, climate_i_s, SUBJ_i_s,
    TEH_n, TEO_n, climate_n, SUBJ_n,
    TEH_f, TEO_f, climate_f, SUBJ_f
]

eval_sets_name = [
    "TEH_i_l", "TEO_i_l", "climate_i_l", "SUBJ_i_l",
    "TEH_i_s", "TEO_i_s", "climate_i_s", "SUBJ_i_s",
    "TEH_n", "TEO_n", "climate_n", "SUBJ_n",
    "TEH_f", "TEO_f", "climate_f", "SUBJ_f"
]


for s, name in zip(eval_sets,eval_sets_name):
    model_eval_gpt3(
    dataset=s,
    num_data=25,
    file_name=name+"_gpt3",
    max_new_tokens=50
)

100%|██████████| 25/25 [00:08<00:00,  2.79it/s]


In [ ]:
big_bench_sets = [modify_the_list, remove_elements, input_independent, add_elements, miscellaneous, Turing]
big_bench_sets_name = [
    "modify_the_list", "remove_elements", "input_independent", "add_elements", "miscellaneous", "Turing"
]

for s, name in zip(big_bench_sets, big_bench_sets_name):
    model_eval_gpt3(
    dataset=s,
    num_data=25,
    file_name=name+"_gpt3",
    max_new_tokens=50,
    big_bench=True,
    big_bench_prompt=big_bench_prompt
)

  1%|          | 1/120 [00:00<01:19,  1.50it/s]

[33, 1]


  2%|▏         | 2/120 [00:01<01:24,  1.39it/s]

[5, 7, 4, 1, 0, 6, 2, 8, 9, 3]


  2%|▎         | 3/120 [00:01<01:08,  1.71it/s]

...

'1'


 97%|█████████▋| 29/30 [00:09<00:00,  3.74it/s]

10110


100%|██████████| 30/30 [00:09<00:00,  3.08it/s]

'00'


# Evaluation using Mistral-7B

In [ ]:
mistral_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
mistral_model = AutoModelForCausalLM.from_pretrained(mistral_model_id,
                                             trust_remote_code=True,
                                             torch_dtype=torch.float16,
                                        attn_implementation="flash_attention_2",
                                        quantization_config=BitsAndBytesConfig(
        load_in_8bit=True,
    ),)
eval_tokenizer = AutoTokenizer.from_pretrained(mistral_model_id)

In [ ]:
eval_sets = [
    TEH_i_l, TEO_i_l, climate_i_l, SUBJ_i_l,
    TEH_i_s, TEO_i_s, climate_i_s, SUBJ_i_s,
    TEH_n, TEO_n, climate_n, SUBJ_n,
    TEH_f, TEO_f, climate_f, SUBJ_f
]

eval_sets_name = [
    "TEH_i_l", "TEO_i_l", "climate_i_l", "SUBJ_i_l",
    "TEH_i_s", "TEO_i_s", "climate_i_s", "SUBJ_i_s",
    "TEH_n", "TEO_n", "climate_n", "SUBJ_n",
    "TEH_f", "TEO_f", "climate_f", "SUBJ_f"
]


for s, name in zip(eval_sets,eval_sets_name):
    model_eval_m(
    dataset=s,
    num_data=25,
    eval_tokenizer=eval_tokenizer,
    model=mistral_model,
    file_name=name+"_m",
    max_new_tokens=50
)

In [ ]:
big_bench_sets = [modify_the_list, remove_elements, input_independent, add_elements, miscellaneous, Turing]
big_bench_sets_name = [
    "modify_the_list", "remove_elements", "input_independent", "add_elements", "miscellaneous", "Turing"
]

for s, name in zip(big_bench_sets, big_bench_sets_name):
    model_eval_m(
    dataset=s,
    num_data=25,
    eval_tokenizer=eval_tokenizer,
    model=mistral_model,
    file_name=name+"_m",
    max_new_tokens=50,
    big_bench=True,
    big_bench_prompt=big_bench_prompt
)

  0%|          | 0/120 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 1/120 [00:08<17:14,  8.69s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 2/120 [00:16<15:55,  8.09s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▎         | 3/120 [00:22<14:17,  7.33s/it]The attention mask and the pad token id were not set. As a consequence, you may

# Evaluation using phi-2

In [ ]:
base_model_id = "microsoft/phi-2"
symbol_model_id = "ChunB1/phi-2_OpenOrca_cot_only_symbol"
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
symbol_model = AutoModelForCausalLM.from_pretrained(symbol_model_id,
                                             trust_remote_code=True,
                                             torch_dtype=torch.float16,
                                        attn_implementation="flash_attention_2",
                                        quantization_config=BitsAndBytesConfig(
        load_in_8bit=True,
    ),)

In [ ]:
symbol_model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiFlashAttention2(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=2560, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
            (lora_dropout): ModuleDict(
  

In [ ]:
model_eval(
    dataset=TEH_i_s,
    num_data=25,
    eval_tokenizer=eval_tokenizer,
    symbol_model=symbol_model,
    file_name="TEH_i_s",
    max_new_tokens=50
)
model_eval(
    dataset=TEO_i_s,
    num_data=25,
    eval_tokenizer=eval_tokenizer,
    symbol_model=symbol_model,
    file_name="TEO_i_s",
    max_new_tokens=50
)
model_eval(
    dataset=climate_i_s,
    num_data=25,
    eval_tokenizer=eval_tokenizer,
    symbol_model=symbol_model,
    file_name="climate_i_s",
    max_new_tokens=50
)
model_eval(
    dataset=SUBJ_i_s,
    num_data=25,
    eval_tokenizer=eval_tokenizer,
    symbol_model=symbol_model,
    file_name="SUBJ_i_s",
    max_new_tokens=50
)

100%|██████████| 25/25 [03:04<00:00,  7.39s/it]


,Input,Target,Symbol Model Generation
0,Question: Is the following sentence subjective...,FOPZ,Output: Yenix\nYenix: The following sentence i...
1,Question: Is the following sentence subjective...,Hibaz,Output: NMQA\nNMQA: The following sentence is ...
2,Question: Is the following sentence subjective...,Nahey,Output: Nahey\nHJZZ: Nahey\nNahey: Nahey\nNahe...
3,Question: Is the following sentence subjective...,KURB,Output: KURB\nQuestion: Is the following sente...
4,Question: Is the following sentence subjective...,34030,Output: 34030\nQuestion: Is the following sent...
5,Question: Is the following sentence subjective...,IMOD,Output: IMOD\nQuestion: Is the following sente...
6,Question: Is the following sentence subjective...,JODV,Output: UETD\nAnswer:\n\nUETD\nUETD\nUETD\nUET...
7,Question: Is the following sentence subjective...,47878,Output: KHHF\nQuestion: Is the following sente...
8,Question: Is the following sentence subjective...,QJQU,Output: YZQS\nQuestion: Is the following sente...
9,Question: Is the following sentence subjective...,MYZE,Output: MYZE\nAnswer:\n\nKagus\nKagus\nMYZE\nM...
